In [ ]:
#| echo: false
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
from score_models.losses import score_matching_loss
from score_models.models.feedforward import FeedForwardModel
from score_models.models.gaussian import GaussianModel
from score_models.data import make_gaussian
from score_models.training import fit, default_optimizer, adam_optimizer
import matplotlib.pyplot as plt

from jax import random, numpy as np, vmap, jacfwd
import optax 

data = make_gaussian()
model = GaussianModel()
model, history = fit(model, data, score_matching_loss, optimizer=adam_optimizer(), steps=600)
plt.plot(history)

In [ ]:
data = make_gaussian()
model = FeedForwardModel()
print(vmap(model)(data).shape, vmap(jacfwd(model))(data).shape)

score_matching_loss(model_func=model, batch=data)

In [ ]:
import equinox as eqx
from jax import grad, nn
dloss = eqx.filter_jit(eqx.filter_grad(score_matching_loss))
# optimizer = optax.chain(
#     optax.clip(0.01),
#     optax.sgd(learning_rate=5e-3),
# )
optimizer = optax.adabelief(learning_rate=1e-3)
# model = GaussianModel()
model = eqx.nn.Sequential(
    [
        eqx.nn.Linear(in_features=1, out_features=1024, key=random.PRNGKey(45)),
        nn.relu(),
        eqx.nn.Linear(in_features=1024, out_features=1, key=random.PRNGKey(39)),
    ]
)
opt_state = optimizer.init(model)
# print(score_matching_loss(model_func=model, batch=data))
# model = eqx.nn.MLP(
#             in_size=1,
#             out_size=1,
#             width_size=1024,
#             depth=1,
#             key=random.PRNGKey(45),
#         )

# opt_state = optimizer.init(model)
# grads = dloss(model, data)
# updates, opt_state = optimizer.update(grads, opt_state)
# model = eqx.apply_updates(model, updates)
# print(score_matching_loss(model_func=model, batch=data))

In [ ]:
dir(updates.submodule)

In [ ]:
updates.mu


In [ ]:
# DEBUG
grads.mu

In [ ]:
data = make_gaussian()
model = GaussianModel()
print(vmap(model)(data).shape, vmap(jacfwd(model))(data).shape)


In [ ]:
import optax 
import equinox as eqx 
from jax.example_libraries import stax
from jax import random, nn

optimizer = optax.adabelief(learning_rate=1e-3)
# model = GaussianModel()
model = eqx.nn.Sequential(
    [
        eqx.nn.Linear(in_features=1, out_features=1024, key=random.PRNGKey(45)),
        nn.relu,  # no problem when commented out
        eqx.nn.Linear(in_features=1024, out_features=1, key=random.PRNGKey(39)),
    ]
)
opt_state = optimizer.init(model)
